In [ ]:
import numpy as np
import pandas as pd

import networkx as nx
import scipy.stats
from statannot import add_stat_annotation
from matplotlib_venn import venn3_unweighted

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

### Define Functions

In [ ]:
def get_degree_centrality(prots, cent_dict):
    cancer_c_arr = list()
    mendelian_c_arr = list()
    other_c_arr = list()
    for unip in prots:
        degree = cent_dict[unip]
        if unip in cancer_genes:
            cancer_c_arr.append([unip, degree])
        if unip in omim_genes:
            mendelian_c_arr.append([unip, degree])  
        if unip not in cancer_genes and unip not in omim_genes:
            other_c_arr.append([unip, degree])

    arr1 = [[unip,degree*(len(prots)-1),'Cancer\ndriver'] for unip,degree in cancer_c_arr]
    arr2 = [[unip,degree*(len(prots)-1),'Mendelian\ndisease'] for unip,degree in mendelian_c_arr]
    arr3 = [[unip,degree*(len(prots)-1),'Other'] for unip,degree in other_c_arr]
    arr_tot = arr1 + arr2 + arr3

    cent_df = pd.DataFrame(arr_tot, columns=['unip','degree','type'])
    return(cent_df)

In [ ]:
def get_centrality(prots, cent_dict):
    cancer_c_arr = list()
    mendelian_c_arr = list()
    other_c_arr = list()
    for unip in prots:
        degree = cent_dict[unip]
        if unip in cancer_genes:
            cancer_c_arr.append([unip, degree])
        if unip in omim_genes:
            mendelian_c_arr.append([unip, degree])  
        if unip not in cancer_genes and unip not in omim_genes:
            other_c_arr.append([unip, degree])

    arr1 = [[unip,degree,'Cancer\ndriver'] for unip,degree in cancer_c_arr]
    arr2 = [[unip,degree,'Mendelian\ndisease'] for unip,degree in mendelian_c_arr]
    arr3 = [[unip,degree,'Other'] for unip,degree in other_c_arr]
    arr_tot = arr1 + arr2 + arr3

    cent_df = pd.DataFrame(arr_tot, columns=['unip','degree','type'])
    return(cent_df)

In [ ]:
%%bash
# create data directory
mkdir -p data
# create symlink to input data
ln -sfn ~/public/networks/NetFeatures-main/data data/NetFeatures
# create output directory
mkdir -p out/NetFeatures

### Read STRING PPI network

The following PPI network is downloaded from the STRING database v11 with the following link: https://stringdb-static.org/download/protein.links.detailed.v11.0/9606.protein.links.detailed.v11.0.txt.gz
Only the experimentally verified interactions with a confidence score higher than 0.4 (400 in the file) are extracted which produced a network of 12811 proteins with 97376 undirected interactions where proteins are listed using Ensembl Protein IDs. Then these ENSP IDs are converted to UniProt IDs which resulted in some duplicates (and some ENSP IDs do not map to UniProt IDs) which resulted in a PPI network of 11851 proteins with 84114 undirected interactions. This network is used for the following analysis.

In [ ]:
string_df = pd.read_table('data/NetFeatures/string_network_experimental_0.4_unips.txt')
string_df = string_df.drop_duplicates()
print(string_df.shape)
string_df.head()

In [ ]:
string_unips = list(np.unique(string_df.values))
print(len(string_unips), string_unips[:5])

### Read 125 known cancer driver genes from Vogelstein et al., 2013

A list of 125 cancer genes is obtained from Vogelstein et al., 2013 (https://pubmed.ncbi.nlm.nih.gov/23539594/) which are listed using HGNC gene symbols. These gene symbols are converted to UniProt protein IDs, which map to 127 proteins used for the following analysis.

In [ ]:
infile = open('data/NetFeatures/cancer_genes_vogelstein_unips.txt','r')
cancer_genes = infile.read().strip().split('\n')
print(len(cancer_genes), cancer_genes[:5])
infile.close()

### Read Mendelian genes from the OMIM database

A list of 4524 Mendelian genes is obtained from the OMIM database and converted from their original HGNC gene symbols to UniProt protein IDs for the following analysis.

In [ ]:
infile = open('data/NetFeatures/omim_genes_unips.txt','r')
omim_genes = infile.read().strip().split('\n')
print(len(omim_genes), omim_genes[:5])
infile.close()

### Look at the overlap between the 3 UniProt protein groups

In [ ]:
print(len(string_unips), string_unips[:5])
print(len(cancer_genes), cancer_genes[:5])
print(len(omim_genes), omim_genes[:5])

In [ ]:
set1 = set(string_unips)
set2 = set(cancer_genes)
set3 = set(omim_genes)

venn3_unweighted([set1, set2, set3], ('STRING', 'Cancer', 'Mendelian'))
plt.show()

### Compute network centrality features

#### Create a graph

In [ ]:
# The following is the size of the STRING PPI network when using UniProt IDs
G = nx.Graph()
G.add_edges_from(string_df.values)
print(len(G.nodes()))
print(len(G.edges()))
prots = list(G.nodes())

In [ ]:
print(len(cancer_genes), len(omim_genes), len(prots))

#### Compute degree centrality

In [ ]:
degree_dict = nx.degree_centrality(G)

In [ ]:
degree_df = get_degree_centrality(prots, degree_dict)
degree_df.head()

In [ ]:
degree_df.to_csv('out/NetFeatures/string_degree.txt',sep='\t',index=False)

#### Compute betweenness centrality

In [ ]:
# takes about 11-12 mins
between_dict = nx.betweenness_centrality(G)

In [ ]:
between_df = get_centrality(prots, between_dict)
between_df.head()

In [ ]:
between_df.to_csv('out/NetFeatures/string_betweenness.txt',sep='\t',index=False)

#### Compute closeness centrality

In [ ]:
# takes about 3-4 mins
close_dict = nx.closeness_centrality(G)

In [ ]:
close_df = get_centrality(prots, close_dict)
close_df.head()

In [ ]:
close_df.to_csv('out/NetFeatures/string_closeness.txt',sep='\t',index=False)

#### Compute eigenvector centrality

In [ ]:
eigen_dict = nx.eigenvector_centrality(G,max_iter=1000)

In [ ]:
eigen_df = get_centrality(prots, eigen_dict)
eigen_df.head()

In [ ]:
eigen_df.to_csv('out/NetFeatures/string_eigenvector.txt',sep='\t',index=False)

### Figure 2

In [ ]:
mpl.rc('figure', titlesize=10)
mpl.rc('axes', labelsize=11)
mpl.rc('xtick', labelsize=9)
mpl.rc('ytick', labelsize=9)

In [ ]:
x = "type"
y = "degree"
order = ['Cancer\ndriver', 'Mendelian\ndisease', 'Other']
flierprops = dict(markerfacecolor = 'lightgrey', markeredgecolor='lightgrey', markersize = 2)

fig = plt.figure(figsize=(8, 2.85))

plt.subplot(1, 4, 1)
df = degree_df.copy()
ax1 = sns.boxplot(x=x, y=y, order=order, data=df, flierprops=flierprops, linewidth=1)
plt.xlabel('')
plt.ylabel('Degree')
plt.xticks(rotation=45)
plt.ylim(0,65)
add_stat_annotation(ax1, data=df, x=x, y=y, order=order,
                    box_pairs=[("Cancer\ndriver", "Other"), ("Mendelian\ndisease", "Other"),
                              ("Cancer\ndriver", "Mendelian\ndisease")],
                    test='Mann-Whitney',
                    text_format='star', loc='outside', verbose=2, linewidth=1,
                    line_offset=0.08, text_offset=0.1)
plt.ylim(0,105)

plt.subplot(1, 4, 2)
df = between_df.copy()
ax2 = sns.boxplot(x=x, y=y, order=order, data=df, flierprops=flierprops, linewidth=1)
plt.xlabel('')
plt.ylabel('Betweenness')
plt.xticks(rotation=45)
plt.ylim(0,0.0031)
add_stat_annotation(ax2, data=df, x=x, y=y, order=order,
                    box_pairs=[("Cancer\ndriver", "Other"), ("Mendelian\ndisease", "Other"),
                              ("Cancer\ndriver", "Mendelian\ndisease")],
                    test='Mann-Whitney',
                    text_format='star', loc='outside', verbose=2, linewidth=1,
                    line_offset=0.08, text_offset=0.1)
plt.ylim(0,0.005)

plt.subplot(1, 4, 3)
df = close_df.copy()
ax3 = sns.boxplot(x=x, y=y, order=order, data=df, flierprops=flierprops, linewidth=1)
plt.xlabel('')
plt.ylabel('Closeness')
plt.xticks(rotation=45)
plt.ylim(0.1,0.32)
add_stat_annotation(ax3, data=df, x=x, y=y, order=order,
                    box_pairs=[("Cancer\ndriver", "Other"), ("Mendelian\ndisease", "Other"),
                              ("Cancer\ndriver", "Mendelian\ndisease")],
                    test='Mann-Whitney',
                    text_format='star', loc='outside', verbose=2, linewidth=1,
                    line_offset=0.08, text_offset=0.1)
plt.ylim(0.1,0.45)

plt.subplot(1, 4, 4)
df = eigen_df.copy()
ax4 = sns.boxplot(x=x, y=y, order=order, data=df, flierprops=flierprops, linewidth=1)
plt.xlabel('')
plt.ylabel('Eigenvector')
plt.xticks(rotation=45)
plt.ylim(0,0.00068)
add_stat_annotation(ax4, data=df, x=x, y=y, order=order,
                    box_pairs=[("Cancer\ndriver", "Other"), ("Mendelian\ndisease", "Other"),
                              ("Cancer\ndriver", "Mendelian\ndisease")],
                    test='Mann-Whitney',
                    text_format='star', loc='outside', verbose=2, linewidth=1,
                    line_offset=0.08, text_offset=0.1)
plt.ylim(0,0.0011)

fig.tight_layout()
fig.subplots_adjust(wspace=.6)
plt.savefig('out/NetFeatures/Fig2.pdf', transparent=True)
plt.show()